## Import Library & API

In [1]:
import pandas as pd
import pandas_ta as ta
from pandas.tseries.offsets import BusinessDay
import pywt
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.graph_objects as go
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.ardl import ARDL, ardl_select_order, UECM
from statsmodels.stats.diagnostic import het_arch
from statsmodels.tools.sm_exceptions import ValueWarning
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
import fredapi as fa
from datetime import date
from twelvedata import TDClient
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Input, LSTM, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import time
import re
import os

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore', ValueWarning)

os.chdir('/Users/fulinq/Documents/KMITL/FinancialEngineering/Y4/Y4T1/PROJECT/ARDL-ECM/Code/Gold/After_Meeting')

In [2]:
fred = fa.Fred(api_key='c948956426006ca126a2dd3bd1f07cee')
td = TDClient(apikey='aa61c51218c248698467af34d09b9d46')

## Data Retrieve ##

In [3]:
def select_column(df, col_name, col_rename = 'close'):
    df_clean = df[[col_name]].copy()

    # Rename column
    df_clean.rename(columns={col_name: col_rename}, inplace=True)
    df_clean.index = pd.to_datetime(df_clean.index).tz_localize(None)
    df_clean = df_clean.sort_index()
    df_clean = df_clean[~df_clean.index.duplicated(keep='first')]
    nan_count_before = df_clean.isna().sum().sum()
    df_clean = df_clean.ffill()
    nan_count_after = df_clean.isna().sum().sum()
    
    print(f"NaN filled: {nan_count_before} -> {nan_count_after}")
    print(f"Total records for {col_name}: {len(df_clean)}")
    print("-" * 30)

    return df_clean

def fetch_twelvedata_10k(td_client, symbol="XAU/USD", col_name="Gold", timezone="Asia/Bangkok", save_csv=True):
    
    # 1. Fetch First Batch (Latest 5000 records)
    ts1 = td_client.time_series(
        symbol=symbol,
        interval="1day",
        outputsize=5000,
        timezone=timezone
    )
    df1 = ts1.as_pandas()
    
    if df1 is None or df1.empty:
        raise ValueError(f"No data returned for {symbol}. Check your API key or Symbol.")
    
    oldest_date = df1.index.min()

    time.sleep(10) 

    ts2 = td_client.time_series(
        symbol=symbol,
        interval="1day",
        outputsize=5000,
        timezone=timezone,
        end_date=oldest_date
    )
    df2 = ts2.as_pandas()

    raw_df = pd.concat([df1, df2])
    
    # Save raw data if requested
    if save_csv:
        filename = f"all_{col_name.lower()}_data.csv"
        raw_df.to_csv(filename)
        print(f"Raw data saved to {filename}")

    return raw_df

def fetch_fred(fred_client, series_id, col_name, percent = False, save_csv=False):
    df = fred_client.get_series(series_id)
    df.index = pd.to_datetime(df.index) 
    print(f'NaN value before processing: {df.isna().sum()}')
    df = df.ffill()
    print(f'NaN value after processing: {df.isna().sum()}')
    df.rename(col_name, inplace=True)
    print(f'Total records for {col_name}: {len(df)}')
    print(f'start date: {df.index.min()}')
    print(f'end date: {df.index.max()}')
    
    if percent:
        df = df.mul(0.01)
        print(f'Total records for {col_name} in percent: {len(df)}')
    
    if save_csv:
        filename = f"all_{col_name.lower()}_data_fred.csv"
        df.to_csv(filename)
        print(f"FRED data saved to {filename}")
    
    return pd.DataFrame(df)

def chow_lin_disaggregate(y_low: pd.Series, X_high: pd.DataFrame,
                          agg_method: str = 'sum', rho: float = None) -> tuple:
    y_low = y_low.dropna().copy()
    X_high = X_high.dropna().copy()
    n_high_per_low = 3  # Quarterly -> Monthly = 3 เดือนต่อไตรมาส

    # หาช่วงเวลาที่ซ้อนทับกัน (Overlapping period)
    quarters = y_low.index
    months = X_high.index
    min_date = max(quarters.min(), months.min().to_period('Q').to_timestamp())
    max_date = min(quarters.max(), months.max().to_period('Q').to_timestamp())

    y_low = y_low[(y_low.index >= min_date) & (y_low.index <= max_date)]
    
    # ปรับช่วงเวลาของ Monthly ให้ครอบคลุม Quarterly พอดี
    month_start = y_low.index.min()
    month_end = (y_low.index.max() + pd.offsets.QuarterEnd()).to_period('M').to_timestamp()
    X_high = X_high[(X_high.index >= month_start) & (X_high.index <= month_end)]

    n_low = len(y_low)
    n_high = n_low * n_high_per_low
    X_high = X_high.iloc[:n_high] # ตัดส่วนเกินออก

    # Build aggregation matrix C (Matrix สำหรับแปลงรายเดือนกลับเป็นไตรมาส)
    C = np.zeros((n_low, n_high))
    for i in range(n_low):
        start_col = i * n_high_per_low
        end_col = start_col + n_high_per_low
        if agg_method == 'sum': # สำหรับ Flow variable เช่น GDP
            C[i, start_col:end_col] = 1.0
        elif agg_method == 'mean': # สำหรับ Stock variable
            C[i, start_col:end_col] = 1.0 / n_high_per_low
        else:
            C[i, end_col - 1] = 1.0

    # Prepare X matrix
    X = X_high.values
    if X.ndim == 1: X = X.reshape(-1, 1)
    X = np.column_stack([np.ones(n_high), X]) # เพิ่ม Intercept

    # OLS เบื้องต้นเพื่อหาค่า Rho (Autocorrelation coefficient)
    X_low = C @ X
    y = y_low.values.flatten()
    beta_ols = np.linalg.lstsq(X_low, y, rcond=None)[0]
    u_low = y - X_low @ beta_ols

    if rho is None: # ถ้าไม่ได้กำหนดมา ให้คำนวณจาก Residuals
        if len(u_low) > 1:
            rho = np.corrcoef(u_low[:-1], u_low[1:])[0, 1]
            rho = np.clip(rho, -0.99, 0.99)
        else:
            rho = 0.0

    # GLS Estimation (พระเอกของงาน)
    # สร้าง Covariance Matrix V ตามโครงสร้าง AR(1)
    V = np.zeros((n_high, n_high))
    for i in range(n_high):
        for j in range(n_high):
            V[i, j] = rho ** abs(i - j)

    V_low = C @ V @ C.T
    try:
        V_low_inv = np.linalg.inv(V_low)
    except:
        V_low_inv = np.linalg.pinv(V_low)

    # คำนวณ Beta ด้วย GLS
    XVX = X_low.T @ V_low_inv @ X_low
    XVy = X_low.T @ V_low_inv @ y
    try:
        beta_gls = np.linalg.solve(XVX, XVy)
    except:
        beta_gls = np.linalg.lstsq(XVX, XVy, rcond=None)[0]

    # คำนวณค่าพยากรณ์และกระจาย Error (Distribute residuals)
    p_high = X @ beta_gls
    u_low_gls = y - X_low @ beta_gls
    VCt = V @ C.T
    
    try:
        dist_matrix = VCt @ np.linalg.inv(V_low)
    except:
        dist_matrix = VCt @ np.linalg.pinv(V_low)

    y_high = p_high + dist_matrix @ u_low_gls # ผลลัพธ์สุดท้าย

    result = pd.Series(y_high, index=X_high.index, name='GDP_Monthly_ChowLin')
    return result, beta_gls, rho

In [4]:
gold = fetch_twelvedata_10k(td, symbol="XAU/USD", col_name="Gold", timezone="Asia/Bangkok", save_csv=True)
gold = select_column(gold, col_name="close", col_rename="Gold")
gold

Raw data saved to all_gold_data.csv
NaN filled: 0 -> 0
Total records for close: 10000
------------------------------


,Gold
datetime,
1988-01-25,469.44000
1988-01-26,470.07001
1988-01-27,465.78000
1988-01-28,466.56000
1988-01-29,454.01001
...,...
2026-02-13,5029.89519
2026-02-14,5043.16482
2026-02-15,5043.20850


In [5]:
dollar_index = fetch_fred(fred, series_id='DTWEXBGS', col_name='Dollar Index')
dollar_index

NaN value before processing: 207
NaN value after processing: 0
Total records for Dollar Index: 5245
start date: 2006-01-02 00:00:00
end date: 2026-02-06 00:00:00


,Dollar Index
2006-01-02,101.4155
2006-01-03,100.7558
2006-01-04,100.2288
2006-01-05,100.2992
2006-01-06,100.0241
...,...
2026-02-02,118.3609
2026-02-03,117.9764
2026-02-04,118.2602
2026-02-05,118.5034


In [6]:
ppi = fetch_fred(fred, series_id='PPIACO', col_name='PPI')
ppi

NaN value before processing: 0
NaN value after processing: 0
Total records for PPI: 1356
start date: 1913-01-01 00:00:00
end date: 2025-12-01 00:00:00


,PPI
1913-01-01,12.100
1913-02-01,12.000
1913-03-01,12.000
1913-04-01,12.000
1913-05-01,11.900
...,...
2025-08-01,262.110
2025-09-01,262.094
2025-10-01,260.724
2025-11-01,261.358


In [7]:
fed_fund = fetch_fred(fred, series_id='FEDFUNDS', col_name='Federal Fund Rate', percent=True)
fed_fund

NaN value before processing: 0
NaN value after processing: 0
Total records for Federal Fund Rate: 859
start date: 1954-07-01 00:00:00
end date: 2026-01-01 00:00:00
Total records for Federal Fund Rate in percent: 859


,Federal Fund Rate
1954-07-01,0.0080
1954-08-01,0.0122
1954-09-01,0.0107
1954-10-01,0.0085
1954-11-01,0.0083
...,...
2025-09-01,0.0422
2025-10-01,0.0409
2025-11-01,0.0388
2025-12-01,0.0372


In [8]:
vix = fetch_fred(fred, series_id='VIXCLS', percent=True,col_name='VIX')
vix['VIX'] = vix['VIX'].mul(1 / np.sqrt(252))
vix

NaN value before processing: 301
NaN value after processing: 0
Total records for VIX: 9423
start date: 1990-01-02 00:00:00
end date: 2026-02-12 00:00:00
Total records for VIX in percent: 9423


,VIX
1990-01-02,0.010860
1990-01-03,0.011459
1990-01-04,0.012107
1990-01-05,0.012668
1990-01-08,0.012763
...,...
2026-02-06,0.011188
2026-02-09,0.010936
2026-02-10,0.011207
2026-02-11,0.011118


In [9]:
unemploy = fetch_fred(fred, series_id='ICSA', col_name='ISCA') #Initial Claims
unemploy

NaN value before processing: 0
NaN value after processing: 0
Total records for ISCA: 3084
start date: 1967-01-07 00:00:00
end date: 2026-02-07 00:00:00


,ISCA
1967-01-07,208000.0
1967-01-14,207000.0
1967-01-21,217000.0
1967-01-28,204000.0
1967-02-04,216000.0
...,...
2026-01-10,199000.0
2026-01-17,210000.0
2026-01-24,209000.0
2026-01-31,232000.0


In [10]:
ip = fetch_fred(fred, series_id='INDPRO', col_name='IP')
ip

NaN value before processing: 0
NaN value after processing: 0
Total records for IP: 1284
start date: 1919-01-01 00:00:00
end date: 2025-12-01 00:00:00


,IP
1919-01-01,4.8739
1919-02-01,4.6585
1919-03-01,4.5238
1919-04-01,4.6046
1919-05-01,4.6315
...,...
2025-08-01,101.5867
2025-09-01,101.7779
2025-10-01,101.5163
2025-11-01,101.9528


In [11]:
gdp = fetch_fred(fred, series_id='GDP', col_name='GDP')
gdp

NaN value before processing: 4
NaN value after processing: 4
Total records for GDP: 319
start date: 1946-01-01 00:00:00
end date: 2025-07-01 00:00:00


,GDP
1946-01-01,NaN
1946-04-01,NaN
1946-07-01,NaN
1946-10-01,NaN
1947-01-01,243.164
...,...
2024-07-01,29511.664
2024-10-01,29825.182
2025-01-01,30042.113
2025-04-01,30485.729


In [12]:
y_target = gdp['GDP']
X_indicator = ip[['IP']]

gdp_monthly_gls, beta, rho = chow_lin_disaggregate(y_low=y_target, X_high=X_indicator, agg_method='sum', rho=None)
print("Estimated Rho (Autocorrelation):", rho)
gdp = gdp_monthly_gls.copy()
gdp_monthly_gls

Estimated Rho (Autocorrelation): 0.99


1947-01-01       77.773508
1947-02-01       80.692880
1947-03-01       84.697612
1947-04-01       78.954153
1947-05-01       82.894249
                  ...     
2025-05-01    10144.558754
2025-06-01    10243.572273
2025-07-01    10350.178201
2025-08-01    10371.135506
2025-09-01    10376.713293
Name: GDP_Monthly_ChowLin, Length: 945, dtype: float64

In [13]:
fed_balance = fetch_fred(fred, series_id='WALCL', col_name='Fed Balance Sheet') #Federal Reserve Total Assets
fed_balance

NaN value before processing: 0
NaN value after processing: 0
Total records for Fed Balance Sheet: 1209
start date: 2002-12-18 00:00:00
end date: 2026-02-11 00:00:00


,Fed Balance Sheet
2002-12-18,719542.0
2002-12-25,732059.0
2003-01-01,730994.0
2003-01-08,723762.0
2003-01-15,720074.0
...,...
2026-01-14,6581700.0
2026-01-21,6584580.0
2026-01-28,6587568.0
2026-02-04,6605909.0


In [14]:
# 1. organize data
realtime_data = {
    'gold': gold,
    'dollar_index': dollar_index,
    'vix': vix,
    'fed_rate': fed_fund,
    'fed_balance': fed_balance,
    'labor_claims': unemploy
}

lagged_data = {
    'ip': ip,
    'gdp': gdp,
    'ppi': ppi
}

# 2. resample & rename
monthly_dfs = []

# process real-time
for name, data in realtime_data.items():
    # FIX: force rename for both Series and DataFrame to match the key (lowercase)
    if isinstance(data, pd.DataFrame):
        data = data.iloc[:, 0].to_frame(name)
    else:
        data = data.to_frame(name)
    
    if name in ['labor_claims', 'vix']:
        monthly_dfs.append(data.resample('ME').mean())
    else:
        monthly_dfs.append(data.resample('ME').last())

# process lagged
for name, data in lagged_data.items():
    if isinstance(data, pd.DataFrame):
        data = data.iloc[:, 0].to_frame(name)
    else:
        data = data.to_frame(name)
    monthly_dfs.append(data.resample('ME').last())

# 3. merge
df_final = pd.concat(monthly_dfs, axis=1)

# 4. handle lag (shift)
vars_to_shift = ['ip', 'ppi']
for col in vars_to_shift:
    df_final[col] = df_final[col].shift(1)
df_final['gdp'] = df_final['gdp'].shift(4)

# 5. target variable
df_final['target_gold'] = df_final['gold'].shift(-1)

# 6. feature selection
features = [
    'gold', 'dollar_index', 'vix', 'fed_rate', 
    'fed_balance', 'labor_claims', 
    'ip', 'gdp','ppi'
]

df_model = df_final[features + ['target_gold']].dropna()

# check
print(f"data range: {df_model.index.min().date()} to {df_model.index.max().date()}")
print(df_model.columns)
df_model

data range: 2006-01-31 to 2026-01-31
Index(['gold', 'dollar_index', 'vix', 'fed_rate', 'fed_balance',
       'labor_claims', 'ip', 'gdp', 'ppi', 'target_gold'],
      dtype='object')


,gold,dollar_index,vix,fed_rate,fed_balance,labor_claims,ip,gdp,ppi,target_gold
2006-01-31,568.95001,99.4311,0.007560,0.0429,828901.0,295750.0,98.0452,4330.637985,163.000,561.75000
2006-02-28,561.75000,99.7695,0.007842,0.0449,840555.0,290750.0,98.1999,4387.339722,164.300,583.65002
2006-03-31,583.65002,100.5600,0.007366,0.0459,833675.0,301750.0,98.2413,4449.366642,161.800,654.50000
2006-04-30,654.50000,98.1412,0.007480,0.0479,844572.0,303600.0,98.4628,4487.497636,162.200,643.70001
2006-05-31,643.70001,97.7705,0.009100,0.0494,851580.0,332750.0,98.7618,4515.041039,164.300,613.09998
...,...,...,...,...,...,...,...,...,...,...
2025-09-30,3869.18000,120.1368,0.009946,0.0422,6608395.0,234750.0,101.5867,10144.558754,262.110,3995.65000
2025-10-31,3995.65000,121.3859,0.011393,0.0409,6587034.0,226750.0,101.7779,10243.572273,262.094,4216.34338
2025-11-30,4216.34338,121.0527,0.012454,0.0388,6552419.0,217600.0,101.5163,10350.178201,260.724,4311.26879
2025-12-31,4311.26879,119.7456,0.009738,0.0372,6640618.0,219000.0,101.9528,10371.135506,261.358,4865.45982


In [15]:
df_model.to_csv('gold_price_model_data.csv')

In [16]:
df_ret = pd.DataFrame()
cols_to_transform = ['gold', 'gdp', 'ip', 'ppi','dollar_index', 'labor_claims', 'fed_balance'] # ไม่เอา IP, PPI ตามแผน Core Model
cols_not_to_transform = ['fed_rate', 'vix'] # ตัวแปรที่ไม่ทำ log return
for col in cols_to_transform:
    if col in df_model.columns:
        df_ret[f'{col}_ret'] = np.log(df_model[col]).diff()
for col in cols_not_to_transform:
    if col in df_model.columns:
        df_ret[f'{col}_change'] = df_model[col].diff()
    
df_ret.dropna(inplace=True)
df_ret

,gold_ret,gdp_ret,ip_ret,ppi_ret,dollar_index_ret,labor_claims_ret,fed_balance_ret,fed_rate_change,vix_change
2006-02-28,-0.012736,0.013008,0.001577,0.007944,0.003398,-0.017051,0.013962,0.0020,0.000282
2006-03-31,0.038245,0.014039,0.000422,-0.015333,0.007892,0.037135,-0.008219,0.0010,-0.000475
2006-04-30,0.114570,0.008533,0.002252,0.002469,-0.024347,0.006112,0.012986,0.0020,0.000113
2006-05-31,-0.016639,0.006119,0.003032,0.012864,-0.003784,0.091680,0.008263,0.0015,0.001620
2006-06-30,-0.048705,0.003576,0.000254,0.009088,0.004875,-0.085442,-0.008424,0.0005,0.001558
...,...,...,...,...,...,...,...,...,...
2025-09-30,0.110673,0.004640,-0.003020,-0.000946,-0.000594,0.020442,0.000759,-0.0011,0.000025
2025-10-31,0.032164,0.009713,0.001880,-0.000061,0.010344,-0.034673,-0.003238,-0.0013,0.001447
2025-11-30,0.053762,0.010353,-0.002574,-0.005241,-0.002749,-0.041190,-0.005269,-0.0021,0.001060
2025-12-31,0.022264,0.002023,0.004291,0.002429,-0.010856,0.006413,0.013371,-0.0016,-0.002716


## Data Preparation ##

In [17]:
df_model = pd.read_csv('gold_price_model_data.csv', index_col=0, parse_dates=True)
df_model

,gold,dollar_index,vix,fed_rate,fed_balance,labor_claims,ip,gdp,ppi,target_gold
2006-01-31,568.95001,99.4311,0.007560,0.0429,828901.0,295750.0,98.0452,4330.637985,163.000,561.75000
2006-02-28,561.75000,99.7695,0.007842,0.0449,840555.0,290750.0,98.1999,4387.339722,164.300,583.65002
2006-03-31,583.65002,100.5600,0.007366,0.0459,833675.0,301750.0,98.2413,4449.366642,161.800,654.50000
2006-04-30,654.50000,98.1412,0.007480,0.0479,844572.0,303600.0,98.4628,4487.497636,162.200,643.70001
2006-05-31,643.70001,97.7705,0.009100,0.0494,851580.0,332750.0,98.7618,4515.041039,164.300,613.09998
...,...,...,...,...,...,...,...,...,...,...
2025-09-30,3869.18000,120.1368,0.009946,0.0422,6608395.0,234750.0,101.5867,10144.558754,262.110,3995.65000
2025-10-31,3995.65000,121.3859,0.011393,0.0409,6587034.0,226750.0,101.7779,10243.572273,262.094,4216.34338
2025-11-30,4216.34338,121.0527,0.012454,0.0388,6552419.0,217600.0,101.5163,10350.178201,260.724,4311.26879
2025-12-31,4311.26879,119.7456,0.009738,0.0372,6640618.0,219000.0,101.9528,10371.135506,261.358,4865.45982


In [18]:
vars_to_log = ['gold', 'dollar_index', 'fed_balance', 'labor_claims', 'ip', 'gdp','ppi', 'target_gold']
for col in vars_to_log:
    df_model[f'ln_{col}'] = np.log(df_model[col])

model_vars = ['fed_rate', 'vix'] + [f'ln_{c}' for c in vars_to_log]
df_ardl = df_model[model_vars].dropna()

df_ardl

,fed_rate,vix,ln_gold,ln_dollar_index,ln_fed_balance,ln_labor_claims,ln_ip,ln_gdp,ln_ppi,ln_target_gold
2006-01-31,0.0429,0.007560,6.343793,4.599465,13.627856,12.597270,4.585429,8.373470,5.093750,6.331057
2006-02-28,0.0449,0.007842,6.331057,4.602863,13.641818,12.580219,4.587005,8.386478,5.101694,6.369302
2006-03-31,0.0459,0.007366,6.369302,4.610755,13.633599,12.617354,4.587427,8.400517,5.086361,6.483872
2006-04-30,0.0479,0.007480,6.483872,4.586407,13.646585,12.623466,4.589679,8.409051,5.088830,6.467233
2006-05-31,0.0494,0.009100,6.467233,4.582623,13.654849,12.715147,4.592711,8.415170,5.101694,6.418528
...,...,...,...,...,...,...,...,...,...,...
2025-09-30,0.0422,0.009946,8.260798,4.788631,15.703851,12.366276,4.620913,9.224693,5.568764,8.292962
2025-10-31,0.0409,0.011393,8.292962,4.798975,15.700614,12.331603,4.622793,9.234406,5.568703,8.346724
2025-11-30,0.0388,0.012454,8.346724,4.796226,15.695345,12.290414,4.620219,9.244759,5.563462,8.368988
2025-12-31,0.0372,0.009738,8.368988,4.785369,15.708716,12.296827,4.624510,9.246782,5.565891,8.489917


In [19]:
def run_adf_test(series, name):
    # Test at Level
    result = adfuller(series.dropna())
    p_value = result[1]
    
    if p_value <= 0.05:
        return f"I(0) - Stationary (p={p_value:.4f})"
    else:
        # ถ้า Level ไม่นิ่ง ให้ลอง Test แบบ Diff (First Difference)
        diff_result = adfuller(series.diff().dropna())
        diff_p_value = diff_result[1]
        
        if diff_p_value <= 0.05:
            return f"I(1) - Stationary at Diff (p={diff_p_value:.4f})"
        else:
            return f"I(2) or Higher (Non-Stationary) (p={diff_p_value:.4f})"
        
summary_data = []
for col in df_ardl.columns:
    status = run_adf_test(df_ardl[col], col)
    summary_data.append({'Variable': col, 'Status': status})

df_status = pd.DataFrame(summary_data)
df_status

,Variable,Status
0,fed_rate,I(1) - Stationary at Diff (p=0.0050)
1,vix,I(0) - Stationary (p=0.0008)
2,ln_gold,I(1) - Stationary at Diff (p=0.0000)
3,ln_dollar_index,I(1) - Stationary at Diff (p=0.0000)
4,ln_fed_balance,I(1) - Stationary at Diff (p=0.0000)
5,ln_labor_claims,I(0) - Stationary (p=0.0182)
6,ln_ip,I(1) - Stationary at Diff (p=0.0000)
7,ln_gdp,I(1) - Stationary at Diff (p=0.0000)
8,ln_ppi,I(1) - Stationary at Diff (p=0.0000)
9,ln_target_gold,I(1) - Stationary at Diff (p=0.0000)


In [20]:
X_cols = ['fed_rate'
          ,'ln_gold'
          ,'ln_dollar_index'
          ,'vix'
          ,'ln_labor_claims'
          ,'ln_ip'
        #   ,'ln_gdp'
          ,'ln_ppi'
        #   ,'ln_fed_balance'
          ]

X = df_ardl[X_cols].dropna()
X = sm.add_constant(X)

vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]

vif_data

,Variable,VIF
0,const,29151.133420
1,fed_rate,1.444787
2,ln_gold,5.066762
3,ln_dollar_index,1.821538
4,vix,1.741195
5,ln_labor_claims,2.576050
6,ln_ip,1.995322
7,ln_ppi,6.128517


In [21]:
train_date_str = '2015-12-31'
df_test_ardl = df_ardl[df_ardl.index <= train_date_str].copy()
df_test_ardl

,fed_rate,vix,ln_gold,ln_dollar_index,ln_fed_balance,ln_labor_claims,ln_ip,ln_gdp,ln_ppi,ln_target_gold
2006-01-31,0.0429,0.007560,6.343793,4.599465,13.627856,12.597270,4.585429,8.373470,5.093750,6.331057
2006-02-28,0.0449,0.007842,6.331057,4.602863,13.641818,12.580219,4.587005,8.386478,5.101694,6.369302
2006-03-31,0.0459,0.007366,6.369302,4.610755,13.633599,12.617354,4.587427,8.400517,5.086361,6.483872
2006-04-30,0.0479,0.007480,6.483872,4.586407,13.646585,12.623466,4.589679,8.409051,5.088830,6.467233
2006-05-31,0.0494,0.009100,6.467233,4.582623,13.654849,12.715147,4.592711,8.415170,5.101694,6.418528
...,...,...,...,...,...,...,...,...,...,...
2015-08-31,0.0014,0.012239,7.033947,4.705577,15.314040,12.525253,4.617324,8.713266,5.267343,7.016610
2015-09-30,0.0014,0.015454,7.016610,4.712509,15.316051,12.504324,4.615484,8.715785,5.256974,7.040869
2015-10-31,0.0012,0.010576,7.040869,4.702369,15.317216,12.495004,4.612689,8.715764,5.242276,6.970185
2015-11-30,0.0012,0.010181,6.970185,4.721530,15.314483,12.500606,4.607842,8.721109,5.233779,6.967250


In [22]:
y_col = 'ln_gold'
X_cols = ['fed_rate'
          ,'ln_dollar_index'
          ,'vix'
          ,'ln_labor_claims'
          ,'ln_ip'
        #   ,'ln_gdp'
          ,'ln_ppi'
        #   ,'ln_fed_balance'
          ]

data_ardl = df_test_ardl[[y_col] + X_cols].dropna()

custom_max_order = {
    'fed_rate': 6,
    'ln_dollar_index': 5,
    'vix': 3,
    'ln_labor_claims': 6,
    'ln_ip': 4,
    'ln_ppi': 4
}

sel_res = ardl_select_order(
    data_ardl[y_col], 
    maxlag=6, 
    exog=data_ardl[X_cols], 
    maxorder=custom_max_order,
    ic='aic'
)

print(f"Best AR Lags: {sel_res.ar_lags}")
print(f"Best DL Orders: {sel_res.dl_lags}")

Best AR Lags: [1, 2, 3, 4, 5, 6]
Best DL Orders: {'fed_rate': [0, 1, 2, 3, 4, 5, 6], 'ln_dollar_index': [0], 'vix': [0], 'ln_labor_claims': [0, 1, 2, 3, 4, 5], 'ln_ip': [0, 1], 'ln_ppi': [0]}


In [23]:
ar_lag = max(sel_res.ar_lags) if isinstance(sel_res.ar_lags, list) else sel_res.ar_lags
dl_lags = {k: (max(v) if isinstance(v, list) else v) for k, v in sel_res.dl_lags.items()}
exog_order = {}
for i in dl_lags:
    exog_order[i] = max(1, dl_lags[i])
    
print(f"\n--- 2. ARDL Levels Analysis & Bounds Test ---")
model_ardl = ARDL(
    data_ardl[y_col], 
    lags=ar_lag, 
    exog=data_ardl[X_cols], 
    order=exog_order
)
res_ardl = model_ardl.fit()
res_ardl.summary()


--- 2. ARDL Levels Analysis & Bounds Test ---


<class 'statsmodels.iolib.summary.Summary'>
"""
                                  ARDL Model Results                                 
=====================================================================================
Dep. Variable:                       ln_gold   No. Observations:                  120
Model:             ARDL(6, 6, 1, 1, 5, 1, 1)   Log Likelihood                 214.343
Method:                      Conditional MLE   S.D. of innovations              0.037
Date:                       Tue, 17 Feb 2026   AIC                           -370.686
Time:                               16:58:20   BIC                           -291.336
Sample:                           07-31-2006   HQIC                          -338.482
                                - 12-31-2015                                         
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 19.2160      5.460      3.520      0.001       8.362      30.069
ln_gold.L1             0.5712      0.101      5.648      0.000       0.370       0.772
ln_gold.L2             0.1596      0.107      1.490      0.140      -0.053       0.373
ln_gold.L3             0.0210      0.103      0.203      0.839      -0.185       0.227
ln_gold.L4            -0.0822      0.108     -0.763      0.448      -0.296       0.132
ln_gold.L5             0.1413      0.108      1.308      0.194      -0.073       0.356
ln_gold.L6             0.0721      0.091      0.793      0.430      -0.109       0.253
fed_rate.L0           -1.0570      4.536     -0.233      0.816     -10.074       7.961
fed_rate.L1           10.1446      7.916      1.282      0.203      -5.591      25.880
fed_rate.L2          -13.7091      7.842     -1.748      0.084     -29.299       1.881
fed_rate.L3           -3.6775      7.568     -0.486      0.628     -18.723      11.368
fed_rate.L4           -5.0089      7.716     -0.649      0.518     -20.347      10.330
fed_rate.L5           18.5718      7.326      2.535      0.013       4.009      33.135
fed_rate.L6          -11.5096      4.085     -2.817      0.006     -19.631      -3.388
ln_dollar_index.L0    -1.9191      0.296     -6.475      0.000      -2.508      -1.330
ln_dollar_index.L1     0.5461      0.365      1.496      0.138      -0.180       1.272
vix.L0                 7.8585      2.318      3.391      0.001       3.251      12.466
vix.L1                -2.2490      2.323     -0.968      0.336      -6.867       2.369
ln_labor_claims.L0    -0.1178      0.147     -0.802      0.425      -0.410       0.174
ln_labor_claims.L1    -0.0148      0.165     -0.089      0.929      -0.343       0.314
ln_labor_claims.L2     0.0718      0.169      0.425      0.672      -0.264       0.408
ln_labor_claims.L3     0.0342      0.163      0.210      0.834      -0.289       0.357
ln_labor_claims.L4    -0.0198      0.155     -0.127      0.899      -0.328       0.289
ln_labor_claims.L5    -0.3848      0.132     -2.921      0.004      -0.647      -0.123
ln_ip.L0               2.2627      0.858      2.638      0.010       0.558       3.968
ln_ip.L1              -2.2734      0.821     -2.768      0.007      -3.906      -0.641
ln_ppi.L0             -1.8507      0.548     -3.378      0.001      -2.940      -0.762
ln_ppi.L1              0.6019      0.546      1.103      0.273      -0.483       1.687
======================================================================================
"""

In [24]:
model_uecm = UECM(
    data_ardl[y_col], 
    lags=6, 
    exog=data_ardl[X_cols], 
    order=exog_order
)
res_uecm = model_uecm.fit()

# 2. รัน Bounds Test จากผลลัพธ์ของ UECM
# case 3 คือมี intercept แต่ไม่มี trend (นิยมใช้ที่สุด)
bt_results = res_uecm.bounds_test(case=3)

print("--- ARDL Bounds Test Results ---")
print(bt_results)

# 3. ดูค่า ECT (ในตาราง summary จะชื่อประมาณ 'diff.ln_gold.L1' หรือตัวแปรที่เป็นระดับ Level)
# หรือดูค่า Adjustment Term โดยตรง
print("\n--- UECM Summary (ดูค่า ECT และนัยสำคัญ) ---")
print(res_uecm.summary())

--- ARDL Bounds Test Results ---
BoundsTestResult
Stat: 3.32913
Upper P-value: 0.0698
Lower P-value: 0.0036
Null: No Cointegration
Alternative: Possible Cointegration


--- UECM Summary (ดูค่า ECT และนัยสำคัญ) ---
                                  UECM Model Results                                 
Dep. Variable:                     D.ln_gold   No. Observations:                  120
Model:             UECM(6, 6, 1, 1, 5, 1, 1)   Log Likelihood                 214.343
Method:                      Conditional MLE   S.D. of innovations              7.029
Date:                       Tue, 17 Feb 2026   AIC                           -370.686
Time:                               16:58:20   BIC                           -291.336
Sample:                           07-31-2006   HQIC                          -338.482
                                - 12-31-2015                                         
                           coef    std err          z      P>|z|      [0.025      0.975]
---------

## ARDL-ECM Forcast

In [25]:
exog_order_pure = {}
for i in exog_order:
    exog_order_pure[i] = [int(j) for j in range(1, exog_order[i]+1)]
ar_order = ar_lag

train_data = df_test_ardl.copy()
test_data = df_ardl[df_ardl.index > train_date_str].copy()

history = train_data.copy()
predictions = []
actuals = test_data[y_col].values

print(f"Train Period: {train_data.index[0].date()} to {train_data.index[-1].date()} (Count: {len(train_data)})")
print(f"Test Period:  {test_data.index[0].date()} to {test_data.index[-1].date()} (Count: {len(test_data)})")
print(f"\nStarting Walk-Forward Forecast (OOS)")

for t in range(len(test_data)):
    model = ARDL(
        endog=history[y_col],
        lags=ar_order,
        exog=history[X_cols],
        order=exog_order_pure,
        trend='c'
    )
    model_fit = model.fit()
    
    next_exog = test_data.iloc[[t]][X_cols]
    
    pred = model_fit.predict(start=len(history), end=len(history), exog_oos=next_exog)
    yhat = pred.values[0]
    predictions.append(yhat)
    
    history = pd.concat([history, test_data.iloc[[t]]])
    
    # warking forward
    # history = history.iloc[1:]
    
    if (t+1) % 12 == 0:
        print(f"Step {t+1}: {test_data.index[t].date()} -> Pred={np.exp(yhat):.4f} | Actual={np.exp(actuals[t]):.4f}")

final_model = ARDL(endog=history[y_col], lags=ar_order, exog=history[X_cols], order=exog_order_pure, trend='c')
final_model_fit = final_model.fit()
next_exog_future = history.iloc[[-1]][X_cols]
pred_future = final_model_fit.predict(start=len(history), end=len(history), exog_oos=next_exog_future)
yhat_future = pred_future.values[0]
# predictions.append(yhat_future)

actual_price = np.exp(actuals)
pred_price = np.exp(predictions)

results = pd.DataFrame({
    'Actual': actuals,
    'Predicted' : predictions,
    'Error' : actuals - predictions,
    'Actual_Price': actual_price,
    'Predicted_Price': pred_price,
    'Error_Price' : actual_price - pred_price
}, index=test_data.index)
last_date = results.index[-1]
next_date = last_date + BusinessDay(n=1)
future_row = pd.DataFrame({
    'Actual': [np.nan],              
    'Predicted': [yhat_future],      
    'Error': [np.nan],               
    'Actual_Price': [np.nan],        
    'Predicted_Price': [np.exp(yhat_future)], 
    'Error_Price': [np.nan]  
}, index=[next_date]) 

results = pd.concat([results, future_row])       
results.round(2)

Train Period: 2006-01-31 to 2015-12-31 (Count: 120)
Test Period:  2016-01-31 to 2026-01-31 (Count: 121)

Starting Walk-Forward Forecast (OOS)
Step 12: 2016-12-31 -> Pred=1196.3330 | Actual=1151.8500
Step 24: 2017-12-31 -> Pred=1249.3369 | Actual=1302.5500
Step 36: 2018-12-31 -> Pred=1183.1620 | Actual=1282.8199
Step 48: 2019-12-31 -> Pred=1507.7775 | Actual=1517.3900
Step 60: 2020-12-31 -> Pred=1788.5531 | Actual=1896.4876
Step 72: 2021-12-31 -> Pred=1795.0848 | Actual=1828.3903
Step 84: 2022-12-31 -> Pred=1747.6701 | Actual=1824.4000
Step 96: 2023-12-31 -> Pred=2061.0911 | Actual=2062.5901
Step 108: 2024-12-31 -> Pred=2752.9766 | Actual=2623.8401
Step 120: 2025-12-31 -> Pred=4264.8011 | Actual=4311.2688


,Actual,Predicted,Error,Actual_Price,Predicted_Price,Error_Price
2016-01-31,7.02,6.96,0.05,1118.04,1058.91,59.13
2016-02-29,7.12,7.01,0.11,1238.14,1107.08,131.06
2016-03-31,7.12,7.12,-0.00,1232.70,1237.77,-5.07
2016-04-30,7.17,7.14,0.03,1293.74,1255.95,37.79
2016-05-31,7.10,7.18,-0.08,1215.24,1313.76,-98.52
...,...,...,...,...,...,...
2025-10-31,8.29,8.27,0.03,3995.65,3886.79,108.86
2025-11-30,8.35,8.30,0.04,4216.34,4038.03,178.31
2025-12-31,8.37,8.36,0.01,4311.27,4264.80,46.47
2026-01-31,8.49,8.39,0.10,4865.46,4421.08,444.38


In [26]:
rmse = np.sqrt(mean_squared_error(actual_price, pred_price))
mae = mean_absolute_error(actual_price, pred_price)

print(f"RMSE (USD): {rmse:.2f}")
print(f"MAE (USD):  {mae:.2f}")

RMSE (USD): 130.26
MAE (USD):  77.67


In [27]:
# plt.figure(figsize=(14, 7))

# plt.axvline(x=pd.to_datetime('2015-12-31'), color='gray', linestyle=':', label='Train/Test Split')

# plt.plot(df_ardl.index, np.exp(df_ardl[y_col]), label='Actual History', color='lightgray')
# plt.plot(test_data.index, actual_price, label='Actual Test Data', color='#1f77b4', linewidth=2)
# plt.plot(test_data.index, pred_price, label='Forecast (Pure OOS)', color='#d62728', linestyle='--', linewidth=2)

# plt.title('Gold Price Forecast: Out-of-Sample Testing (2016-Present)')
# plt.xlabel('Date')
# plt.ylabel('Price (USD)')
# plt.legend()
# plt.grid(True, alpha=0.3)
# plt.show()

In [28]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_ardl.index, 
    y=np.exp(df_ardl[y_col]),
    mode='lines',
    name='Actual History',
    line=dict(color='lightgray')
))

fig.add_trace(go.Scatter(
    x=test_data.index, 
    y=actual_price,
    mode='lines',
    name='Actual Test (2016-Present)',
    line=dict(color='#1f77b4', width=2)
))

fig.add_trace(go.Scatter(
    x=test_data.index, 
    y=pred_price,
    mode='lines',
    name='Forecast',
    line=dict(color='#d62728', width=2, dash='dash')
))

fig.update_layout(
    width=1000,
    height=700,
    autosize=False,
    title='Gold Price Forecast: Interactive Walk-Forward Validation',
    yaxis_title='Price (USD)',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(count=5, label="5y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(visible=True),
        type="date"
    ),
    template="plotly_white",
    legend=dict(x=0, y=1)
)

fig.show()

## Technical Data

In [45]:
def denoise_data(data, wavelet='db4', level=2):
    coeff = pywt.wavedec(data, wavelet, mode="per")
    sigma = (1/0.6745) * np.median(np.abs(coeff[-1] - np.median(coeff[-1])))
    uthesh = sigma * np.sqrt(2 * np.log(len(data)))
    new_coeff = [coeff[0]]
    for i in coeff[1:]:
        new_coeff.append(pywt.threshold(i, value=uthesh, mode='soft'))
    reconstructed = pywt.waverec(new_coeff, wavelet, mode='per')
    return reconstructed[:len(data)]

In [46]:
macro_feature = results[['Predicted']].copy()
macro_feature.columns = ['Macro_Signal']

current_date = pd.Timestamp.now().normalize()
trading_days_index = pd.date_range(start=macro_feature.index.min(), end=current_date, freq='B')
macro_daily = macro_feature.reindex(trading_days_index)

macro_daily = macro_feature.resample('D').asfreq()
macro_daily = macro_daily.fillna(method='ffill')
macro_daily

,Macro_Signal
2016-01-31,6.964999
2016-02-01,6.964999
2016-02-02,6.964999
2016-02-03,6.964999
2016-02-04,6.964999
...,...
2026-01-29,8.358151
2026-01-30,8.358151
2026-01-31,8.394139
2026-02-01,8.394139


In [47]:
df_daily = pd.read_csv('all_gold_data.csv', index_col=0, parse_dates=True)
df_daily.sort_index(inplace=True)
df_daily = df_daily[~df_daily.index.duplicated(keep='first')]
df_daily['actual_close'] = df_daily['close'].copy()
df_daily['ln_close'] = np.log(df_daily['close'])

for col in ['open', 'high', 'low', 'close']:
    df_daily[col] = denoise_data(df_daily[col].values)

# Momentum & Trend
df_daily.ta.rsi(length=14, append=True)
df_daily.ta.macd(fast=12, slow=26, signal=9, append=True)
df_daily.ta.adx(length=14, append=True)
df_daily.ta.cci(length=20, append=True)

# Volatility & Bands
df_daily.ta.bbands(length=20, std=2, append=True)
df_daily.ta.atr(length=14, append=True)

# Moving Average Distances
df_daily.ta.ema(length=50, append=True)
df_daily.ta.ema(length=200, append=True)
df_daily['dist_ema50'] = (df_daily['close'] - df_daily['EMA_50']) / df_daily['EMA_50']
df_daily['dist_ema200'] = (df_daily['close'] - df_daily['EMA_200']) / df_daily['EMA_200']

# Statistical & Others
df_daily['daily_range'] = (df_daily['high'] - df_daily['low']) / df_daily['open']
rolling_mean = df_daily['close'].rolling(window=20).mean()
rolling_std = df_daily['close'].rolling(window=20).std()
df_daily['z_score'] = (df_daily['close'] - rolling_mean) / rolling_std

cols_to_drop = ['EMA_50', 'EMA_200', 'BBU_20_2.0', 'BBL_20_2.0', 'BBM_20_2.0']
df_daily.drop(columns=[c for c in cols_to_drop if c in df_daily.columns], inplace=True)
df_daily.dropna(inplace=True)
df_daily

,open,high,low,close,actual_close,ln_close,RSI_14,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,...,DMP_14,DMN_14,CCI_20_0.015,BBB_20_2.0,BBP_20_2.0,ATRr_14,dist_ema50,dist_ema200,daily_range,z_score
datetime,,,,,,,,,,,,,,,,,,,,,
1988-10-31,416.706300,416.297781,414.617686,415.125277,412.16000,6.021412,66.428702,0.343328,0.929851,-0.586523,...,20.631899,10.446973,99.685720,3.174463,0.840382,2.397508,-0.001948,-0.052864,0.004032,1.327054
1988-11-01,417.318441,417.002843,415.306386,415.699957,411.04999,6.018715,68.605654,0.560384,0.917526,-0.357141,...,21.593265,9.853395,113.478782,3.022438,0.858227,2.360369,-0.000544,-0.051066,0.004065,1.396628
1988-11-02,418.202558,418.032685,416.249035,416.463764,421.32999,6.043416,71.272066,0.784986,0.913702,-0.128716,...,23.186753,9.157240,141.715473,2.855995,0.894219,2.358394,0.001242,-0.048855,0.004265,1.536949
1988-11-03,419.208833,419.218319,417.311256,417.305855,421.06000,6.042775,73.903642,1.019187,0.918322,0.100865,...,24.823633,8.402336,175.328971,2.675614,0.941802,2.386692,0.003138,-0.046487,0.004549,1.722462
1988-11-04,419.886897,420.024967,418.036231,417.873031,421.06000,6.042775,75.529554,1.236307,0.908354,0.327953,...,25.213799,7.725313,185.331691,2.502893,0.969101,2.410436,0.004324,-0.044761,0.004736,1.828892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026-02-13,4949.528746,5048.900603,4896.065265,5012.787300,5029.89519,8.523154,56.374670,73.027543,-11.487498,84.515041,...,20.822229,24.702325,2.915510,10.908538,0.547767,162.725643,0.053094,0.204611,0.030879,0.186232
2026-02-14,5029.154119,5043.924247,5021.100735,5037.660183,5043.16482,8.525789,57.550236,72.422922,-9.673696,82.096617,...,20.520193,24.344007,33.638601,10.937352,0.587953,153.326451,0.055904,0.208057,0.004538,0.342903
2026-02-15,5024.826956,5028.433482,5030.851235,5037.559909,5043.20850,8.525798,57.543503,71.115884,-8.784587,79.900471,...,20.425643,24.231838,32.388095,10.957669,0.585682,143.033611,0.053574,0.205538,-0.000481,0.334051


In [48]:
# """
# For joining daily-marco variable to technical data
# """

# macro_to_merge = dollar_index.join(vix, how='inner')
# macro_to_merge = macro_to_merge[macro_to_merge.index >= '2016-02-01']
# macro_to_merge

In [49]:
df_final = macro_daily.join(df_daily, how='right')
# df_final = df_final.join(macro_to_merge, how='left')
df_final = df_final.ffill()
df_final = df_final.dropna()
forecast_horizon = 5
for i in range(1, forecast_horizon + 1):
    col_name = f'target_return_{i}d'
    df_final[col_name] = np.log(df_final['actual_close']).shift(-i) - np.log(df_final['close'])

df_predict_latest = df_final.tail(forecast_horizon).copy()

# threshold = 0.00
# choices = [1, 0]
# for i in range(1, forecast_horizon + 1):
#     target_col = f'target_return_{i}d'
#     signal_col = f'signal_{i}d'
    
#     conditions = [
#         (df_final[target_col] >= threshold),
#         (df_final[target_col] < -threshold)
#     ]
    
#     df_final[signal_col] = np.select(conditions, choices, default=0)


df_final.to_csv('gold_technical.csv')
df_final

,Macro_Signal,open,high,low,close,actual_close,ln_close,RSI_14,MACD_12_26_9,MACDh_12_26_9,...,ATRr_14,dist_ema50,dist_ema200,daily_range,z_score,target_return_1d,target_return_2d,target_return_3d,target_return_4d,target_return_5d
datetime,,,,,,,,,,,,,,,,,,,,,
2016-02-01,6.964999,1129.180653,1134.835866,1122.156735,1136.095369,1128.84998,7.028955,91.591037,12.726878,3.503100,...,13.967130,0.034001,0.005313,0.011229,1.952461,-0.006159,0.005998,0.017144,0.032555,0.047314
2016-02-02,6.964999,1136.648239,1143.521974,1129.152465,1143.859025,1129.12000,7.029194,92.972820,13.965452,3.793340,...,13.995871,0.039393,0.012060,0.012642,2.075493,-0.000812,0.010333,0.025745,0.040503,0.038579
2016-02-03,6.964999,1144.999140,1153.329503,1137.071430,1152.387046,1142.93005,7.041350,94.116490,15.456993,4.227904,...,14.157457,0.045210,0.019407,0.014199,2.141341,0.002905,0.018317,0.033075,0.031151,0.038392
2016-02-04,6.964999,1154.945497,1165.316342,1146.630734,1162.370512,1155.73999,7.052496,95.118153,17.245833,4.813395,...,14.480896,0.052027,0.027949,0.016179,2.179824,0.009691,0.024449,0.022525,0.029766,0.070191
2016-02-05,6.964999,1165.234088,1179.340865,1156.946851,1173.995803,1173.68994,7.067908,95.977044,19.378185,5.556598,...,15.046119,0.059948,0.037836,0.019218,2.207514,0.014498,0.012573,0.019815,0.060239,0.053124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026-02-13,8.501178,4949.528746,5048.900603,4896.065265,5012.787300,5029.89519,8.523154,56.374670,73.027543,-11.487498,...,162.725643,0.053094,0.204611,0.030879,0.186232,0.006042,0.006050,-0.004063,-0.017261,NaN
2026-02-14,8.501178,5029.154119,5043.924247,5021.100735,5037.660183,5043.16482,8.525789,57.550236,72.422922,-9.673696,...,153.326451,0.055904,0.208057,0.004538,0.342903,0.001101,-0.009013,-0.022211,NaN,NaN
2026-02-15,8.501178,5024.826956,5028.433482,5030.851235,5037.559909,5043.20850,8.525798,57.543503,71.115884,-8.784587,...,143.033611,0.053574,0.205538,-0.000481,0.334051,-0.008993,-0.022191,NaN,NaN,NaN


In [50]:
df_final.columns

Index(['Macro_Signal', 'open', 'high', 'low', 'close', 'actual_close',
       'ln_close', 'RSI_14', 'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9',
       'ADX_14', 'DMP_14', 'DMN_14', 'CCI_20_0.015', 'BBB_20_2.0',
       'BBP_20_2.0', 'ATRr_14', 'dist_ema50', 'dist_ema200', 'daily_range',
       'z_score', 'target_return_1d', 'target_return_2d', 'target_return_3d',
       'target_return_4d', 'target_return_5d'],
      dtype='object')

## Preparation Before TCN (Wavelet)

In [ ]:
# def denoise_data(data, wavelet='db4', level=2):
#     coeff = pywt.wavedec(data, wavelet, mode="per")
#     sigma = (1/0.6745) * np.median(np.abs(coeff[-1] - np.median(coeff[-1])))
#     uthesh = sigma * np.sqrt(2 * np.log(len(data)))
#     new_coeff = [coeff[0]]
#     for i in coeff[1:]:
#         new_coeff.append(pywt.threshold(i, value=uthesh, mode='soft'))
#     reconstructed = pywt.waverec(new_coeff, wavelet, mode='per')
#     return reconstructed[:len(data)]

# df = pd.read_csv('gold_technical.csv', index_col=0, parse_dates=True)
# df['close_denoised'] = denoise_data(df['close'].values)

In [70]:
# plt.figure(figsize=(15, 6))
# plt.plot(df['close'][-100:], label='Original Noisy Price', alpha=0.5, color='gray')
# plt.plot(df['close_denoised'][-100:], label='Wavelet Denoised (Trend)', color='blue', linewidth=2)
# plt.title('Gold Price: Original vs Denoised')
# plt.legend()
# plt.show()

In [ ]:
# fig = go.Figure()

# fig.add_trace(go.Scatter(
#     x=df.index, 
#     y=df['close'],
#     mode='lines',
#     name='Actual Close',
#     line=dict(color='lightgray'),
#     hovertemplate='Actual: $%{y:.2f}<br>Date: %{x}<extra></extra>'
# ))

# fig.add_trace(go.Scatter(
#     x=df.index, 
#     y=df['close_denoised'],
#     mode='lines',
#     name='Denoised Close',
#     line=dict(color='#d62728', width=1.5),
#     hovertemplate='Denoised: $%{y:.2f}<br>Date: %{x}<extra></extra>'
# ))

# fig.update_layout(
#     width=1000,
#     height=700,
#     title='Gold Price: Original vs Denoised (Dynamic Scale)',
#     template="plotly_white",
    
#     yaxis=dict(
#         title='Price (USD)',
#         range=[df['close'].min() * 0.95, df['close'].max() * 1.05],
#         fixedrange=False,
#         autorange=True
#     ),
#     xaxis=dict(
#         title='Date',
#         type="date",
#         rangeslider=dict(
#             visible=True,
#             thickness=0.05
#         ),
#         rangeselector=dict(
#             buttons=list([
#                 dict(count=1, label="1y", step="year", stepmode="backward"),
#                 dict(count=5, label="5y", step="year", stepmode="backward"),
#                 dict(step="all")
#             ])
#         )
#     ),
#     uirevision='constant', 
#     legend=dict(x=0.01, y=0.99, bgcolor='rgba(255,255,255,0.5)'),
#     margin=dict(l=50, r=50, t=80, b=50)
# )

# fig.update_yaxes(matches=None)
# fig.show()

## CNN-LSTM

In [60]:
df = pd.read_csv('gold_technical.csv', parse_dates=['datetime'], index_col='datetime')

df_train_full = df.dropna(subset=['target_return_5d']).copy()

feature_cols = [c for c in df.columns if 'target' not in c and c != 'actual_close' and c != 'close']
target_cols = ['target_return_1d', 'target_return_2d', 'target_return_3d', 'target_return_4d', 'target_return_5d']

X = df_train_full[feature_cols].values
y = df_train_full[target_cols].values

train_size = int(len(X) * 0.8)
X_train_raw, X_test_raw = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train_raw)
X_test_scaled = scaler_X.transform(X_test_raw)

def create_sequences(X, y, window_size=30):
    Xs, ys = [], []
    for i in range(len(X) - window_size):
        Xs.append(X[i:(i + window_size)])
        ys.append(y[i + window_size])
    return np.array(Xs), np.array(ys)

WINDOW_SIZE = 30
X_train, y_train_seq = create_sequences(X_train_scaled, y_train, WINDOW_SIZE)
X_test, y_test_seq = create_sequences(X_test_scaled, y_test, WINDOW_SIZE)

print(f"Training Shape: {X_train.shape}, Target Shape: {y_train_seq.shape}")

model = Sequential([
    Conv1D(filters=64, kernel_size=2, activation='swish', input_shape=(X_train.shape[1], X_train.shape[2])),
    MaxPooling1D(pool_size=2),
    
    LSTM(100, return_sequences=False, activation='tanh'),
    Dropout(0.2),
    
    # Output Layer: Regression 5 value (1-5 days)
    Dense(50, activation='swish'),
    Dense(5) # no Activation (Linear) for Regression
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
model.summary()

history = model.fit(
    X_train, y_train_seq,
    epochs=50,
    batch_size=32,
    validation_data=(X_test, y_test_seq),
    verbose=1
)

# Real-world Inference (Prediction)
# Use the latest 30 days of data from df as input for the model
last_30_days_features = df[feature_cols].tail(WINDOW_SIZE).values
last_30_days_scaled = scaler_X.transform(last_30_days_features).reshape(1, WINDOW_SIZE, len(feature_cols))

prediction_returns = model.predict(last_30_days_scaled)
print(f"Predicted Returns (1-5 days): {prediction_returns[0]}")

Training Shape: (2133, 30, 20), Target Shape: (2133, 5)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_3 (Conv1D)               │ (None, 29, 64)         │         2,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 14, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 100)            │        66,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 5)              │           255 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 73,929 (288.79 KB)

 Trainable params: 73,929 (288.79 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0102 - mae: 0.0675 - val_loss: 0.0041 - val_mae: 0.0446
Epoch 2/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 7.9940e-04 - mae: 0.0218 - val_loss: 0.0025 - val_mae: 0.0374
Epoch 3/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 5.2589e-04 - mae: 0.0175 - val_loss: 0.0021 - val_mae: 0.0358
Epoch 4/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 3.7264e-04 - mae: 0.0148 - val_loss: 0.0017 - val_mae: 0.0305
Epoch 5/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2.8256e-04 - mae: 0.0129 - val_loss: 0.0012 - val_mae: 0.0255
Epoch 6/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2.7154e-04 - mae: 0.0124 - val_loss: 0.0012 - val_mae: 0.0258
Epoch 7/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2.3291e-04 - mae: 0.0116 - val_loss: 8.8001e-04 - val_mae: 0.0219
Epoch 8/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.3012e-04 - mae: 0.0114 - val_loss: 9.4529e-04 - val_mae: 0.0233
Epoch 9/50
67/67 ━━━━━━━━━━━━━━━━━━━

In [61]:
last_date = df.index[-1]
next_5_trading_days = [last_date + BusinessDay(n) for n in range(1, 6)]

prediction_returns = model.predict(last_30_days_scaled)[0]
current_actual_price = df['actual_close'].iloc[-1]
predicted_prices = [current_actual_price * np.exp(ret) for ret in prediction_returns]

print("\n" + "="*55)
print(f"📅 ราคาปิดล่าสุด ({last_date.strftime('%Y-%m-%d')}): {current_actual_price:.2f}")
print(f"🔮 พยากรณ์ 5 วันทำการข้างหน้า:")
print("-" * 55)

for i, (price, date) in enumerate(zip(predicted_prices, next_5_trading_days), 1):
    # เทียบราคาอนาคตกับราคาปัจจุบัน (เปลี่ยนเป็นเทียบกับวันก่อนหน้าก็ได้ตามชอบ)
    trend = "🟢" if price > current_actual_price else "🔴"
    diff = price - current_actual_price
    
    # แสดงผล: วันที่ | ราคาพยากรณ์ | เทรนด์ | ส่วนต่าง
    print(f"Day {i} ({date.strftime('%Y-%m-%d')}): {price:.2f}  {trend} ({diff:+.2f})")

print("="*55)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step

📅 ราคาปิดล่าสุด (2026-02-17): 4927.00
🔮 พยากรณ์ 5 วันทำการข้างหน้า:
-------------------------------------------------------
Day 1 (2026-02-18): 4931.86  🟢 (+4.86)
Day 2 (2026-02-19): 4849.36  🔴 (-77.65)
Day 3 (2026-02-20): 5021.83  🟢 (+94.83)
Day 4 (2026-02-23): 4944.22  🟢 (+17.22)
Day 5 (2026-02-24): 5030.62  🟢 (+103.62)


In [62]:
y_pred_test = model.predict(X_test)

test_start_index = train_size + WINDOW_SIZE
base_prices_test = df_train_full['close'].iloc[test_start_index:].values

min_len = min(len(y_pred_test), len(base_prices_test))
y_pred_clipped = y_pred_test[:min_len]
y_test_clipped = y_test[:min_len] # y_test มาจากขั้นตอน Split ก่อนหน้านี้
base_prices_clipped = base_prices_test[:min_len].reshape(-1, 1)

all_actual_prices = []
all_pred_prices = []
rmse_results = {}

print("\n" + "="*50)
print(f"📊 สรุปผลความแม่นยำ (RMSE Price Basis)")
print("="*50)

for i in range(5):
    true_price = base_prices_clipped * np.exp(y_test_clipped[:, i].reshape(-1, 1))
    pred_price = base_prices_clipped * np.exp(y_pred_clipped[:, i].reshape(-1, 1))

    all_actual_prices.append(true_price)
    all_pred_prices.append(pred_price)

    rmse = np.sqrt(mean_squared_error(true_price, pred_price))
    rmse_results[f'{i+1}d'] = rmse
    
    print(f"RMSE {i+1}d : ${rmse:.2f}")

flat_actual = np.concatenate(all_actual_prices).flatten()
flat_pred = np.concatenate(all_pred_prices).flatten()
combined_rmse = np.sqrt(mean_squared_error(flat_actual, flat_pred))

print("-" * 50)
print(f"📉 Combined RMSE (1-5d): ${combined_rmse:.2f}")
print("="*50)

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 

📊 สรุปผลความแม่นยำ (RMSE Price Basis)
RMSE 1d : $45.31
RMSE 2d : $83.63
RMSE 3d : $79.94
RMSE 4d : $128.67
RMSE 5d : $131.76
--------------------------------------------------
📉 Combined RMSE (1-5d): $99.35
